In [ ]:
using BenchmarkTools
using SparseArrays
using Distributed
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

: 

	frag_face_channel(in_chan, out_chan, V, EV, FE, sp_idx)

Parallel fragmentation of faces in `FE` against faces in `sp_idx`.


In [ ]:
function frag_face_channel(in_chan, out_chan, V, EV, FE, sp_idx)
    run_loop = true
    while run_loop 
        
        sigma = take!(in_chan)
        
        if sigma != -1
            put!(out_chan,Lar.Arrangement.frag_face(V, EV, FE, sp_idx, sigma))
        else
            run_loop = false
        end
    end
    return V,EV
end

: 

In [ ]:
V = Float64[
    0 0 0; 0 1 0;
    1 1 0; 1 0 0;
    0 0 1; 0 1 1;
    1 1 1; 1 0 1
]


: 

In [ ]:
EV = sparse(Int8[
    -1  1  0  0  0  0  0  0;
    0 -1  1  0  0  0  0  0;
    0  0 -1  1  0  0  0  0;
    -1  0  0  1  0  0  0  0;
    -1  0  0  0  1  0  0  0;
    0 -1  0  0  0  1  0  0;
    0  0 -1  0  0  0  1  0;
    0  0  0 -1  0  0  0  1;
    0  0  0  0 -1  1  0  0;
    0  0  0  0  0 -1  1  0;
    0  0  0  0  0  0 -1  1;
    0  0  0  0 -1  0  0  1;
])


: 

In [ ]:
FE = sparse(Int8[
    1  1  1 -1  0  0  0  0  0  0  0  0;
    0  0  0  0  0  0  0  0 -1 -1 -1  1;
    -1  0  0  0  1 -1  0  0  1  0  0  0;
    0 -1  0  0  0  1 -1  0  0  1  0  0;
    0  0 -1  0  0  0  1 -1  0  0  1  0;
    0  0  0  1 -1  0  0  1  0  0  0 -1;
])

: 

In [ ]:
fs_num = size(FE, 1)
    
FV = Lar.compute_FV( EV, FE )
model = (convert(Lar.Points,V'), FV)
sp_idx = Lar.spaceindex(model)

in_chan = Distributed.RemoteChannel(()->Channel{Int64}(fs_num*2))
out_chan = Distributed.RemoteChannel(()->Channel{Tuple}(10))


@async begin
    for sigma2 in 1:fs_num
        put!(in_chan, sigma2)
    end
    for p in Distributed.workers()
        put!(in_chan, -1)
    end
    for p in Distributed.workers()
        @async Distributed.remote_do(
        frag_face_channel, p, in_chan, out_chan, V, EV, FE, sp_idx)
    end
end

#@time frag_face_channel(in_chan, out_chan, V, EV, FE, sp_idx)
#@btime frag_face_channel(in_chan, out_chan, V, EV, FE, sp_idx)

: 

In [ ]:
function inizia()
    fs_num = size(FE, 1)
    
    FV = Lar.compute_FV( EV, FE )
    model = (convert(Lar.Points,V'), FV)
    sp_idx = Lar.spaceindex(model)

    in_chan = Distributed.RemoteChannel(()->Channel{Int64}(fs_num*2))
    out_chan = Distributed.RemoteChannel(()->Channel{Tuple}(10))

    @async begin
        for sigma2 in 1:fs_num
            put!(in_chan, sigma2)
        end
        for p in Distributed.workers()
            put!(in_chan, -1)
        end
        @sync for p in Distributed.workers() 
            #modificato Base.remote_do in Distributed.remote_do altrimenti non chiamava la funzione
            @async Distributed.remote_do(
            frag_face_channel, p, in_chan, out_chan, V, EV, FE, sp_idx)
        end
    end
end

: 

In [ ]:
out_chan

: 

: 

: 